https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f

https://exceptionshub.com/selenium-cant-click-element-because-other-element-obscures-it.html

https://www.toolsqa.com/selenium-webdriver/handle-iframes-in-selenium/

https://www.selenium.dev/documentation/webdriver/elements/finders/

https://www.selenium.dev/selenium/docs/api/javascript/module/selenium-webdriver/index_exports_By.html

In [1]:
import time, datetime, requests,random, os, hashlib

from pathlib import Path

from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
#//*** install xlsxwriter for pandas
#pip install xlsxwriter

#//*********************************************************
#//*** Build the tgt_date from the slected Quarter and Year
#//*********************************************************
tgt_year = 2022
quarter = "Q2"

if quarter == "Q1":
    tgt_date = f"{tgt_year}-01-01"
elif quarter == "Q2": 
    tgt_date = f"{tgt_year}-04-01"
elif quarter == "Q3": 
    tgt_date = f"{tgt_year}-07-01"
elif quarter == "Q4": 
    tgt_date = f"{tgt_year}-10-01"

tgt_date = datetime.datetime.strptime(tgt_date, "%Y-%m-%d")

print(tgt_date)

#//*** Get Filename for dataframe Cache
cache_filepath =  f"{tgt_year}_{quarter}_scripts.dat"
current_dir = Path(os.getcwd()).absolute()
cache_filepath = current_dir.joinpath(cache_filepath)

#//*** If Cache File Exists Load it. Else Start a Fresh DataFrame
if os.path.exists(cache_filepath):
    cache_df = pd.read_pickle(cache_filepath)
else:
    cache_df = pd.DataFrame()




2022-04-01 00:00:00


In [2]:
#//*** Parses a 
def get_scripts(source_urls,headless=True):
    
    out = []
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        page_driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        page_driver = webdriver.Firefox()

    
    
    for i,url in enumerate(source_urls):
        
        print(f"({i+1}/{len(source_urls)}) - {url}")
        try:
            page_driver.get(url)
        except:
            print("Problem Getting Page...Skipping")
            continue
            
        
        try:
        
            loop_story = {
                "url" : url,
                "headline" : page_driver.find_elements(By.CLASS_NAME,"headline")[0].text,
                "time_text" : page_driver.find_elements(By.CLASS_NAME,"lastmodified")[0].text,
                "body_text" : page_driver.find_elements(By.CLASS_NAME,"body-text")[0].text,
            }
            #//*** Split the Date
            loop_story['time_date'] = loop_story['time_text'].split()

            #//*** Rebuild Date string using only the first four fields. Drop the extra time
            loop_story['time_date'] = " ".join(loop_story['time_date'][:4])

            loop_story['time_date'] = datetime.datetime.strptime(loop_story['time_date'], "%A, %B %d, %Y")
        except:
            print("Problem Processing Story. Skipping")
            continue
        out.append(loop_story)
        
        #wait between 1/2 to 3 seconds
        time.sleep(random.randint(500, 3000)/1000)                                                              

    page_driver.quit()
                                                                        
    return out


In [3]:
    
    
def get_all_urls_in_page(source_url,tgt_date,headless=True):

    #//*** Keep all classnames in a dictionary
    g = {
        #//*** Classname for the Show More Button
        "class_more_button" : "show-button-more",
        
        #//*** Grid Classname that contains the linked stories
        "story_grid" : "grid3",
        
        #//*** Story Classes that contain the href Links
        "story_link_class" : "AnchorLink",
        
    }
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        driver = webdriver.Firefox()


    # Load Web Page
    driver.get(source_url)
    
    # execute script to scroll down the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")


    #//*****************************************************
    #//*** Wait until the Show More Button is loaded
    #//*****************************************************
    elements = []
    control = 0
    while len(elements) == 0:

        #//*** Page is Loaded when class_more_button is displayed
        #elements = driver.find_elements_by_class_name(g["class_more_button"])
        elements = driver.find_elements(By.CLASS_NAME,g["class_more_button"])

        control += 1
        #//*** Wait if element not found
        if len(elements) == 0:
            time.sleep(1)

        if control > 15:
            print("Show More Button Not Loaded!")
            return
            break

    #//*************************************************
    #//*** The first elements is the button to click
    #//*************************************************
    elem_show_more = elements[0]

    #actions = ActionChains(driver)
    #actions.move_to_element(element).perform()

    print(elem_show_more.is_displayed())
    print(elem_show_more.location['y'])
    
    #results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])
    results = driver.find_elements(By.CLASS_NAME,g["story_grid"])[0].find_elements(By.CLASS_NAME,g["story_link_class"])

    if len(results) > 0:
        print("Last Page: ", results[0].get_property('href'))

    
    
    #print(results[-1].get_property('href'))

    
    #print(story_time,tgt_date, story_time < tgt_date)
                                                                        
    #//*** Get the last url to check in stories are within the quarter
    last_url = results[-1].get_property('href')
    
    try:
        #//*** Download the last script
        last_script = get_scripts([last_url])[0]
    except:
        print("Problem getting last script")
        print("Last url: ", last_url)
        print("get Scripts: ", get_scripts([last_url]))
        print("Skipping..")
        return []
    last_date = last_script['time_date']
    print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
    print("Last Script: ", last_script['time_date'] < tgt_date)
    
    #//*** Maximum number of while loops.
    maxDepth = 20
    depth = 0
    
    #//*** Click Show More until Last Story is older than the tgt_date
    while last_date > tgt_date:
        depth += 1
        
        #//*** Scroll to Bottom of page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        
        print(f"Clicking More Depth: {depth}")
        #print(dir(elem_show_more))
        while elem_show_more.is_displayed() == False:
            print("Is Displayed:",str(elem_show_more.is_displayed()))
            time.sleep(1)
        
        
        try:
            #//*** Click Show More
            elem_show_more.click()
        except:
            iframes = driver.find_elements_by_tag_name("iframe")
            for iframe in iframes:
                print(iframe.get_property("name"))
                
                
                driver.execute_script("arguments[0].style.visibility='hidden'", iframe)
            #//*** Click Show More
            elem_show_more.click()
                
        #    return

    
        #//*** Get Updated Results
        #results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])        
        results = driver.find_elements(By.CLASS_NAME,g["story_grid"])[0].find_elements(By.CLASS_NAME,g["story_link_class"])        
        
        #//*** Get the last url to check in stories are within the quarter
        last_url = results[-1].get_property('href')

        #//*** Download the last script
        last_script = get_scripts([last_url])[0]
        last_date = last_script['time_date']
        print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
        print("Last Script: ", last_script['time_date'] < tgt_date)

        if depth > maxDepth:
            print("Maximum Depth Reached on Clicking Show More")
            break
        
    
    out = []
    
    for result in results:
        out.append(result.get_property('href'))
    
    #//*** Shut down main scrape
    driver.quit()
    
    

    return out




In [4]:
#//*** Hash url list and compare to cache_df
#//*** returns a tuple of urls (orig_urls,dupe_urls)
def keep_original_urls(urls,cache_df):
    orig_urls = []
    dupe_urls = []
    
    tdf = pd.DataFrame()
    tdf['urls'] = urls
    
    tdf['hash'] = pd.util.hash_pandas_object(tdf['urls'])


    #//*** Check for Duplicates
    tdf['dupe'] = tdf['hash'].isin(cache_df['hash'])
    #tdf['dupe'] = tdf['urls'].isin(cache_df['urls'])
    
    #//*** Keep Only False Stories, which are not duplicates
    #//*** This means a story can have only a single category.
    #//*** May have to handle multiple categories at a future time
    #print(f"Length Before: {len(tdf)} ")

    orig_urls = list(tdf[tdf['dupe'] == False]['urls'])
    dupe_urls = list(tdf[tdf['dupe'] == True]['urls'])

    
    
    return orig_urls,dupe_urls

In [5]:

url_items = [
    {
        "url" : "https://abc7news.com/tag/climate-change/",
        "cat" : "Climate"
    },
    {
        "url" : "https://abc7news.com/tag/environment/",
        "cat" : "Environment"
    },
 
    {
        "url" : "https://abc7news.com/tag/economy/",
        "cat" : "Economy"
    },
    {
        "url" : "https://abc7news.com/tag/jobs/",
        "cat" : "Jobs"
    },    
    {
        "url" : "https://abc7news.com/tag/safety/",
        "cat" : "Safety"
    },
    {
        "url" : "https://abc7news.com/tag/health/",
        "cat" : "Health"
    },
    {
        "url" : "https://abc7news.com/education/",
        "cat" : "Education"
    },
    {
        "url" : "https://abc7news.com/tag/race-and-culture/",
        "cat" : "Race"
    },
    {
        "url" : "https://abc7news.com/tag/discrimination/",
        "cat" : "Discrimination"
    },
    {
        "url" : "https://abc7news.com/tag/civil-rights/",
        "cat" : "Civil Rights"
    },
    {
        "url" : "https://abc7news.com/tag/building-a-better-bay-area/",
        "cat" : "BABBA"
    },
    {
        "url" : "https://abc7news.com/7onyourside/",
        "cat" : "7OYS"
    },
    {
        "url" : "https://abc7news.com/tag/covid-19/",
        "cat" : "COVID"
    },

    {
        "url" : "https://abc7news.com/iteam/",
        "cat" : "I-TEAM"
    },
    

    
]
urlpages = ['https://abc7news.com/education/',
            'https://abc7news.com/tag/building-a-better-bay-area/',
            'https://abc7news.com/7onyourside/',
            'https://abc7news.com/tag/covid-19/',
            'https://abc7news.com/tag/climate-change/',
            'https://abc7news.com/iteam/'
           ]

#urlpage ='https://abc7news.com/education/'
#urls = get_all_urls_in_page(urlpages[1],tgt_date,False)
#print(urlpage)

#//*** Loop through top level url_items
for item in url_items:
    cat_url = item['url']
    cat = item['cat']
    
    #//*** Download the ctageory URLs
    urls = get_all_urls_in_page(cat_url,tgt_date,False)
    
    #//*** Get non-duplicate urls to download and a list of the duplicate urls
    #//*** We may need to incorporate duplicates with different categories...later
    if len(cache_df) > 0:
        urls_to_dl,dupe_urls = keep_original_urls(urls,cache_df)
    else:
        #//*** There is no cache initialize and move one
        urls_to_dl = urls
        dupe_urls = []
    print("=====================")
    print(f"Category: {cat}")
    print("=====================")
    print(f"DL Items: {len(urls_to_dl)} - Duplicate: {len(dupe_urls)}")
    
    if len(urls_to_dl) == 0:
        print("No New Items to Download!")
    
    if len(urls_to_dl) > 0:
        scripts = get_scripts(urls_to_dl)
        
        #//*** Convert Scripts to DataFrame
        df = pd.DataFrame()
        #df['urls'] = urls_to_dl

        s = {}

        #//*** Get each script value as a list
        for script in scripts:

            for key,value in script.items():
                if key not in s.keys():
                    s[key] = []
                s[key].append(value)

        #//*** build Urls field
        #//*** Use the Story URL in case an url was skipped.
        df['urls'] = s['url']
        df['hash'] = pd.util.hash_pandas_object(df['urls'])
        df['cat'] = cat

        #//*** Add lists of script values as columns
        for key,value in s.items():
            df[key] = value

        #//*** Combine cache_df and df
        cache_df = pd.concat([cache_df,df])


#//*** Everything is Gathered. Write Cache_df to disk
#//*** Write DF to file
pd.to_pickle(cache_df,cache_filepath)
    

True
1766
Last Page:  https://abc7news.com/coldplay-climate-pledge-arena-music-of-the-spheres-tour-carbon-neutral-levis-stadium-2022/11824183/
(1/1) - https://abc7news.com/polar-bear-earth-day-month-disneynature/11766027/
Last Script:  Disneynature's 'Polar Bear' sheds light on challenges beloved animals face due to climate change 
 Tuesday, April 19, 2022
Last Script:  False
Clicking More Depth: 1
(1/1) - https://abc7news.com/polar-bear-earth-day-month-disneynature/11766027/
Last Script:  Disneynature's 'Polar Bear' sheds light on challenges beloved animals face due to climate change 
 Tuesday, April 19, 2022
Last Script:  False
Clicking More Depth: 2
(1/1) - https://abc7news.com/rain-california-drought-2022-san-jose-weather-western/11571287/
Last Script:  Some San Jose residents could run out of water by summer if drought continues, expert says 
 Wednesday, February 16, 2022
Last Script:  True
Category: Climate
DL Items: 36 - Duplicate: 0
(1/36) - https://abc7news.com/coldplay-climat

(24/36) - https://abc7news.com/bluebonnets-texas-wildflowers-where-to-find/11765466/
(25/36) - https://abc7news.com/planet-possible-earth-month-national-geographic-disney/11744400/
(26/36) - https://abc7news.com/wei-tai-kwok-climate-change-solutions-reality-project-bay-area-earth-day/11717986/
(27/36) - https://abc7news.com/mattel-carbon-neutral-eco-friendly-toys-sustainable-playsets/11713324/
(28/36) - https://abc7news.com/dolly-parton-national-geographic-travel-amy-alipio-earth-month-2022/11717885/
(29/36) - https://abc7news.com/wildfire-smoke-california-wildfires-climate-change-noaa/11699092/
(30/36) - https://abc7news.com/branching-out-a-nat-geo-earth-day-adventure-trailer-ginger-zee-national-geographic/11764727/
(31/36) - https://abc7news.com/stanford-study-methane-emission-atmosphere-fighting-pollution/11693880/
(32/36) - https://abc7news.com/uc-berkeley-biothermal-storage-borehole-heat-chambers-cal-all-electric-campus-carbon-free-2035/11693934/
(33/36) - https://abc7news.com/pio

(5/10) - https://abc7news.com/google-campus-san-jose-downtown-west-new-center/11748998/
(6/10) - https://abc7news.com/san-mateo-union-highs-school-district-stay-in-the-bay-with-100k-six-figure-jobs-trade/11743205/
(7/10) - https://abc7news.com/california-four-day-work-week-bill-4-ca-mark-takano/11725277/
(8/10) - https://abc7news.com/walmart-truck-driver-salary-jobs-drivers-trucking/11725367/
(9/10) - https://abc7news.com/the-great-resignation-2022-turnover-rate-covid-return-to-work/11688711/
(10/10) - https://abc7news.com/great-resignation-2022-statistics-san-francisco-bay-area-businesses-staffing-crisis/11681368/
True
1797
Last Page:  https://abc7news.com/school-safety-violence-in-schools-shooting-resources-bay-area/3165609/
(1/1) - https://abc7news.com/sfpd-staffing-shortages-san-francisco-police-department-crime-in-sf-officers-bay-area/11746659/
Last Script:  San Francisco police chief explains what's driving dire staffing issues; nearly 600 officers short 
 Thursday, April 14, 202

(15/35) - https://abc7news.com/judy-sakaki-sonoma-state-university-patrick-mccallum-scandal/11834308/
(16/35) - https://abc7news.com/haley-taylor-schlitz-smu-dedman-school-of-law-dallas-texas/11831498/
(17/35) - https://abc7news.com/anthony-anderson-graduation-howard-university-blackish-phylicia-rashad/11831790/
(18/35) - https://abc7news.com/superintendents-21st-century-awards-scholarship-program-san-francisco-scholarships-for-sfusd-students-sf/11780333/
(19/35) - https://abc7news.com/stanford-university-climate-school-john-doerr-donation-of-sustainability/11817740/
(20/35) - https://abc7news.com/shanthi-gonzales-ousd-oakland-school-board-member-resigns-unified-district-merging/11812678/
(21/35) - https://abc7news.com/oakland-teachers-strike-friday-school-closures-one-day-district-ousd-oea/11803532/
(22/35) - https://abc7news.com/high-paying-internships-paid-summer-student/11790226/
(23/35) - https://abc7news.com/university-of-california-native-american-students-tuition-waiver-fees/11

(9/11) - https://abc7news.com/pge-discrimination-lawsuit-filed-employee-suing-for-hostile-working-environment-emanuel-lankford/11425740/
(10/11) - https://abc7news.com/nob-hill-gazette-san-francisco-piedmont-real-estate-article/11252171/
(11/11) - https://abc7news.com/ca-reparations-reparation-taskforce-racial-wealth-gap-black/11123530/
True
1797
Last Page:  https://abc7news.com/bloody-sunday-1965-selma-march-john-lewis-voting-rights/11625943/
(1/1) - https://abc7news.com/movement-black-lives-matter-freedom-march/10330623/
Last Script:  Trio of young, Black female activist leaders inspire change through 'Freedom March NYC' 
 Sunday, February 28, 2021
Last Script:  True
Category: Civil Rights
DL Items: 12 - Duplicate: 0
(1/12) - https://abc7news.com/bloody-sunday-1965-selma-march-john-lewis-voting-rights/11625943/
(2/12) - https://abc7news.com/freedom-to-vote-act-voting-rights-bill-mlk-day/11480932/
(3/12) - https://abc7news.com/martin-luther-king-jr-day-mlk-in-bay-area-2022/11480322/
(

(37/72) - https://abc7news.com/earth-day-2022-east-oakland-students-plant-trees-for-neighborhood/11775467/
(38/72) - https://abc7news.com/genesis-8-smash-gg-twitch-san-jose-convention-center/11754002/
(39/72) - https://abc7news.com/coronavirus-mask-mandates-bay-area-covid-cases-case-rate/11752785/
(40/72) - https://abc7news.com/google-campus-san-jose-downtown-west-new-center/11748998/
(41/72) - https://abc7news.com/bay-area-public-transit-bart-schedule-muni/11746302/
(42/72) - https://abc7news.com/ba-2-variant-omicron-marin-county-8th-grade-students-covid/11745189/
(43/72) - https://abc7news.com/san-mateo-union-highs-school-district-stay-in-the-bay-with-100k-six-figure-jobs-trade/11743205/
(44/72) - https://abc7news.com/richmond-mayor-tom-butt-homeless-population-housing/11742563/
(45/72) - https://abc7news.com/oakland-vaccine-mandate-oakand-coliseum-colisuem-authority/11742240/
(46/72) - https://abc7news.com/san-francisco-ambrosia-bakery-sf-pruning-trees-illegal/11738671/
(47/72) - ht

(1/1) - https://abc7news.com/gavin-newsom-covid-tests-positive-19-vaccine-paxlovid/11904259/
Last Script:  Gov. Newsom tests positive for COVID-19 
 Saturday, May 28, 2022
Last Script:  False
Clicking More Depth: 1
(1/1) - https://abc7news.com/gavin-newsom-covid-tests-positive-19-vaccine-paxlovid/11904259/
Last Script:  Gov. Newsom tests positive for COVID-19 
 Saturday, May 28, 2022
Last Script:  False
Clicking More Depth: 2
(1/1) - https://abc7news.com/covid-surge-california-coronavirus-cases-san-francisco-bay-area-omicron-variant-in-ca/11787541/
Last Script:  COVID cases up 50% in California since March, CDC says 
 Monday, April 25, 2022
Last Script:  False
Clicking More Depth: 3
(1/1) - https://abc7news.com/covid-surge-california-coronavirus-cases-san-francisco-bay-area-omicron-variant-in-ca/11787541/
Last Script:  COVID cases up 50% in California since March, CDC says 
 Monday, April 25, 2022
Last Script:  False
Clicking More Depth: 4
(1/1) - https://abc7news.com/europe-covid-surg

(3/36) - https://abc7news.com/california-wildfires-pge-fire-victim-settlement-medical-liens-held-back-from-wildfire-payments-ca-victims/11897275/
(4/36) - https://abc7news.com/chicago-mob-lake-mead-bodies-found/11870686/
(5/36) - https://abc7news.com/freeway-shooting-bay-area-shootings-580-oakland/11871107/
(6/36) - https://abc7news.com/sf-firefighter-attack-robert-muhammad-gabriel-shin-sffd-jeanine-nicholson/11865971/
(7/36) - https://abc7news.com/jessie-rush-norcal-militia-steven-carrillo-air-force-grizzly-scouts-boogaloo-movement/11861594/
(8/36) - https://abc7news.com/ukraine-flag-vandalized-sf-home-vandal-ukrainian-hate-crime/11858841/
(9/36) - https://abc7news.com/san-francisco-elevators-california-elevator-inspections-cal-osha-safety-permits/11846578/
(10/36) - https://abc7news.com/california-elevator-inspections-cal-osha-san-francisco-elevators-safety-permits/11841532/
(11/36) - https://abc7news.com/sf-crime-burglary-plea-deal-chesa-boudin/11833000/
(12/36) - https://abc7news.c

In [6]:
#print(last_url)

In [7]:
#cache_df['time_date'] = cache_df['time_date'].dt.date
#print("Length Before de-dupe:",len(cache_df)
cache_df = cache_df.drop_duplicates(subset="url")
      
print("Length Before de-dupe:",len(cache_df)

SyntaxError: invalid syntax (Temp/ipykernel_23336/4157665526.py, line 3)

In [9]:
#//*** Write to excel file
output_filename = f"{tgt_year}_{quarter}_Collected_Stories.xlsx"
print(output_filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')

#//*** Build Sheet and Data from From each url_item element
for url_item in url_items:
    sheet = url_item['cat']
    
    tdf = cache_df[cache_df['cat'] == sheet].copy()
    
    tdf = tdf[['urls','time_date','time_text','headline','body_text']].sort_values('time_date',ascending=False)
    
    tdf.to_excel(writer,sheet_name=sheet)
    
try:
    writer.save()    
except:
    print("Trouble Saving the Spreadsheet. It's Probably Open Somewhere. Close it and Retry")    
    

2022_Q2_Collected_Stories.xlsx


In [ ]:
cache_df

In [ ]:
cache_df.drop_duplicates(subset="url")

In [ ]:
url_items = [
    {
        "url" : "https://abc7news.com/tag/climate-change/",
        "cat" : "Climate"
    },
    {
        "url" : "https://abc7news.com/tag/environment/",
        "cat" : "Environment"
    },
 
    {
        "url" : "https://abc7news.com/tag/economy/",
        "cat" : "Economy"
    },
    {
        "url" : "https://abc7news.com/tag/jobs/",
        "cat" : "Jobs"
    },    
    {
        "url" : "https://abc7news.com/tag/safety/",
        "cat" : "Safety"
    },
    {
        "url" : "https://abc7news.com/tag/health/",
        "cat" : "Health"
    },
    {
        "url" : "https://abc7news.com/education/",
        "cat" : "Education"
    },
    {
        "url" : "https://abc7news.com/tag/race-and-culture/",
        "cat" : "Race"
    },
    {
        "url" : "https://abc7news.com/tag/discrimination/",
        "cat" : "Discrimination"
    },
    {
        "url" : "https://abc7news.com/tag/civil-rights/",
        "cat" : "Civil Rights"
    },
    {
        "url" : "https://abc7news.com/tag/building-a-better-bay-area/",
        "cat" : "BABBA"
    },
    {
        "url" : "https://abc7news.com/7onyourside/",
        "cat" : "7OYS"
    },
    {
        "url" : "https://abc7news.com/tag/covid-19/",
        "cat" : "COVID"
    },

    {
        "url" : "https://abc7news.com/iteam/",
        "cat" : "I-TEAM"
    },
    

    
]

cache_df['time_date'] = cache_df['time_date'].dt.date

#//*** Write to excel file
output_filename = f"{tgt_year}_{quarter}_Collected_Stories.xlsx"
print(output_filename)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')

#//*** Build Sheet and Data from From each url_item element
for url_item in url_items:
    sheet = url_item['cat']
    
    tdf = cache_df[cache_df['cat'] == sheet].copy()
    
    tdf = tdf[['urls','time_date','time_text','headline','body_text']]
    
    tdf.to_excel(writer,sheet_name=sheet)
    
try:
    writer.save()    
except:
    print("Trouble Saving the Spreadsheet. It's Probably Open Somewhere. Close it and Retry")   